In [80]:
!pip install google-generativeai langchain langchain-google-genai langgraph chromadb pymed Bio

!pip install -Uq "google-genai==1.7.0"

In [48]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import google.generativeai as genai
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [81]:
from kaggle_secrets import UserSecretsClient
from Bio import Entrez

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_key")
PubMed_API = UserSecretsClient().get_secret('PubMed')

genai.configure(api_key=GOOGLE_API_KEY)

In [68]:
GEMINI_API_KEY = GOOGLE_API_KEY
#PPLX_API_KEY = os.getenv("PPLX_API_KEY") # Perplexity API Key

# --- Klucze API Narzędzi ---
FIRECRAWL_API_KEY = os.getenv("FIRECLAWL_API_KEY") 
PUBMED_API_KEY = PubMed_API 

# --- Inne Konfiguracje ---
# PubMed (Entrez) wymaga adresu email
PUBMED_EMAIL = "wiktortobota13@Gmail.com" 


print(f"GEMINI_API_KEY loaded: {'Yes' if GEMINI_API_KEY else 'No'}")
print(f"FIRECRAWL_API_KEY loaded: {'Yes' if FIRECRAWL_API_KEY else 'No'}")
print(f"PUBMED_EMAIL set to: {PUBMED_EMAIL}")

# Możesz dodać tu logikę wyboru domyślnego LLM, jeśli chcesz
DEFAULT_LLM_PROVIDER = "gemini" if GEMINI_API_KEY else "openai" if OPENAI_API_KEY else None
print(f"Default LLM Provider based on available keys: {DEFAULT_LLM_PROVIDER}")

GEMINI_API_KEY loaded: Yes
FIRECRAWL_API_KEY loaded: No
PUBMED_EMAIL set to: wiktortobota13@Gmail.com
Default LLM Provider based on available keys: gemini


In [64]:
Entrez.email   = PUBMED_EMAIL
Entrez.api_key = PUBMED_API_KEY

In [66]:

def generate_completion(prompt: str, model_preference: str = "gemini", max_tokens: int = 1000) -> str:
    """
    Generuje uzupełnienie tekstu używając preferowanego dostawcy LLM.

    Args:
        prompt (str): Tekst wejściowy dla LLM.
        model_preference (str): Preferowany dostawca ('gemini' lub 'openai').
        max_tokens (int): Maksymalna liczba tokenów w odpowiedzi.

    Returns:
        str: Wygenerowany tekst lub komunikat błędu.
    """

    try:
        # Wybierz odpowiedni model Gemini (np. gemini-pro)
        model = genai.GenerativeModel('gemini-1.5-flash-002')
        response = model.generate_content(prompt)
        # Sprawdź czy odpowiedź zawiera tekst (może być zablokowana)
        if response.parts:
             return response.text
        else:
             # Sprawdź przyczynę braku odpowiedzi (np. blokada bezpieczeństwa)
             print(f"Ostrzeżenie: Odpowiedź Gemini nie zawiera tekstu. Powód: {response.prompt_feedback}")
             return f"Błąd: Odpowiedź Gemini była pusta lub zablokowana ({response.prompt_feedback})."
    except Exception as e:
        print(f"Błąd podczas komunikacji z Gemini API: {e}")
        return f"Błąd Gemini API: {e}"



# --- Przykładowe użycie (można uruchomić ten plik bezpośrednio do testów) ---
if __name__ == '__main__':
    test_prompt = "Napisz krótki opis działania sieci neuronowej."
    print("\nTestowanie generowania LLM:")

    print("\n--- Test Gemini ---")
    gemini_response = generate_completion(test_prompt, model_preference="gemini")
    print(gemini_response)


Testowanie generowania LLM:

--- Test Gemini ---
Sieć neuronowa to system obliczeniowy inspirowany strukturą i działaniem ludzkiego mózgu. Składa się z połączonych ze sobą węzłów (neuronów) ułożonych w warstwach: wejściowej, ukrytych i wyjściowej.  Informacja w postaci danych wejściowych jest przekazywana przez sieć, gdzie każdy neuron przetwarza ją, stosując wagę przypisaną do każdej połączenia.  Te wagi są modyfikowane w procesie uczenia, w którym sieć dostosowuje swoje parametry na podstawie porównania wyników z danymi oczekiwanymi.  Dzięki temu sieć uczy się rozpoznawać wzorce i przewidywać wyniki na podstawie nowych, nieznanych danych.  Im więcej danych treningowych i im lepiej zaprojektowana sieć, tym dokładniejsze są jej przewidywania.



In [74]:
#tu chwilowo nie mam API
from typing import List, Dict, Any, Optional

def search_firecrawl(query: str, fetch_content: bool = True, max_results: int = 5) -> List[Dict[str, Any]]:
    """Wykonuje wyszukiwanie za pomoc¹ Firecrawl."""
    if not firecrawl_client:
        print("B³¹d: Klient Firecrawl jest nieaktywny.")
        return []
    try:
        print(f"Wysy³anie zapytania do Firecrawl: {query}")
        # Wywo³anie API Firecrawl - BEZ limitowania wyników tutaj, jeœli API go nie wspiera bezpoœrednio
        # (parametr max_results tutaj jest tylko dla informacji, nie u¿ywamy go do krojenia poni¿ej)
        results = firecrawl_client.search(
            query,
            params={'pageOptions': {'fetchPageContent': fetch_content}}
        )
        # SprawdŸ typ wyniku na wszelki wypadek
        if not isinstance(results, list):
             print(f"Ostrze¿enie: Firecrawl API zwróci³o nieoczekiwany typ danych ({type(results)}), oczekiwano listy.")
             # Spróbuj obs³u¿yæ, jeœli to mo¿liwe, lub zwróæ pust¹ listê
             if isinstance(results, dict) and 'data' in results and isinstance(results['data'], list):
                  results = results['data'] # Próba odzyskania listy z typowej struktury API
             else:
                  return [] # Zwróæ pust¹ listê, jeœli nie mo¿na przetworzyæ

        print(f"Otrzymano {len(results)} wyników z Firecrawl dla zapytania '{query}'")

        formatted_results = []
        # Iteruj po WSZYSTKICH wynikach zwróconych przez API
        for res in results: # <--- Iterujemy po results_list
            if isinstance(res, dict):
                # U¿yj 'title' jeœli jest, inaczej spróbuj 'name' (standardowo Firecrawl zwraca 'title')
                # U¿yj 'description' jeœli jest, inaczej spróbuj 'snippet'
                # U¿yj 'markdown' lub 'content' jeœli jest, inaczej None
                title = res.get('title') or res.get('name')
                snippet = res.get('description') or res.get('snippet')
                content = None
                if fetch_content:
                    content = res.get('markdown') or res.get('content') # Dodaj inne mo¿liwe klucze treœci, jeœli znasz

                formatted_results.append({
                    'url': res.get('url'),
                    'title': title,
                    'snippet': snippet,
                    'content': content,
                    'source': 'firecrawl_search'
                })
            else:
                 print(f"Ostrze¿enie: Pomiêto nieprawid³owy element wyniku Firecrawl: {res}")

        return formatted_results
    except Exception as e:
        print(f"B³¹d podczas wyszukiwania w Firecrawl dla zapytania '{query}': {e}")
        # Dodajmy traceback dla lepszego debugowania
        import traceback
        traceback.print_exc()
        return []

# --- PubMed (Entrez) Client ---
try:
    from Bio import Entrez
    Entrez.email = PUBMED_EMAIL
    if not Entrez.email or Entrez.email == "wiktortobota13@gmail.com":
        print("Ostrze¿enie: Nie ustawiono prawid³owego adresu email dla PubMed Entrez w .env (PUBMED_EMAIL).")
        entrez_active = False
    else:
         print(f"Klient PubMed Entrez skonfigurowany z emailem: {Entrez.email}")
         entrez_active = True

except ImportError:
    Entrez = None
    entrez_active = False
    print("Ostrze¿enie: Biblioteka biopython nie jest zainstalowana. PubMed Entrez nie bêdzie dzia³aæ.")


def search_pubmed_entrez(query: str, max_results: int = 5) -> Optional[List[Dict[str, Any]]]:
    """Wyszukuje w PubMed u¿ywaj¹c Bio.Entrez."""
    if not entrez_active or not Entrez:
        print("Nie mo¿na u¿yæ PubMed API (Entrez): Klient nieaktywny lub brak biblioteki/emaila.")
        return None # Zwróæ None, aby wskazaæ problem z t¹ metod¹

    try:
        print(f"Wysy³anie zapytania do PubMed (Entrez): {query}")
        handle = Entrez.esearch(db="pubmed", term=query, retmax=str(max_results), api_key=PUBMED_API_KEY) # Dodano opcjonalny klucz API
        record = Entrez.read(handle)
        handle.close()
        pmids = record["IdList"]

        if not pmids:
            print("PubMed (Entrez): Nie znaleziono artyku³ów.")
            return [] # Zwróæ pust¹ listê, jeœli nie ma wyników

        print(f"PubMed (Entrez): Znaleziono {len(pmids)} ID artyku³ów.")
        handle = Entrez.efetch(db="pubmed", id=pmids, rettype="medline", retmode="xml", api_key=PUBMED_API_KEY) # Dodano opcjonalny klucz API
        records = Entrez.read(handle)
        handle.close()

        articles = []
        pubmed_articles = records.get('PubmedArticle', [])
        if isinstance(pubmed_articles, dict): pubmed_articles = [pubmed_articles]

        for article_data in pubmed_articles:
             medline_citation = article_data.get('MedlineCitation')
             if not medline_citation: continue
             article_info = medline_citation.get('Article', {})
             title = article_info.get('ArticleTitle', 'Brak tytu³u')
             abstract_dict = article_info.get('Abstract', {})
             abstract = abstract_dict.get('AbstractText', [''])[0]

             doi = None
             article_ids = article_info.get('ELocationID', [])
             if not isinstance(article_ids, list): article_ids = [article_ids]
             for identifier in article_ids:
                  if identifier.attributes.get('EIdType') == 'doi':
                       doi = str(identifier)
                       break
             if not doi:
                  pubmed_data = article_data.get('PubmedData', {})
                  article_id_list = pubmed_data.get('ArticleIdList', [])
                  if isinstance(article_id_list, dict): article_id_list = article_id_list.get('ArticleId', [])
                  if not isinstance(article_id_list, list): article_id_list = [article_id_list]
                  for identifier in article_id_list:
                       if identifier.attributes.get('IdType') == 'doi':
                            doi = str(identifier)
                            break

             pmid = medline_citation.get('PMID', '')
             authors_list = article_info.get('AuthorList', [])
             authors = []
             if authors_list:
                  for author_entry in authors_list:
                       if isinstance(author_entry, dict):
                            lastname = author_entry.get('LastName', '')
                            forename = author_entry.get('ForeName', '')
                            if lastname and forename: authors.append(f"{forename} {lastname}")

             articles.append({
                 'title': str(title), 'authors': authors, 'abstract': str(abstract),
                 'doi': doi, 'pmid': str(pmid),
                 'url': f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/" if pmid else None,
                 'source': 'pubmed_entrez'
             })
        print(f"PubMed (Entrez): Pomyœlnie przetworzono {len(articles)} artyku³ów.")
        return articles

    except Exception as e:
        print(f"B³¹d podczas wyszukiwania w PubMed (Entrez): {e}")
        # Mo¿na logowaæ szczegó³y b³êdu
        # Sprawdzenie specyficznych b³êdów Entrez (np. 429 Too Many Requests)
        if "HTTP Error 429" in str(e):
             print("B³¹d Entrez: Przekroczono limit zapytañ. Spróbuj ponownie póŸniej.")
        return None # Zwróæ None w przypadku b³êdu

Klient PubMed Entrez skonfigurowany z emailem: wiktortobota13@Gmail.com


In [75]:
from typing import List, Dict, Any


class WebSearchAgent:
    def _generate_queries(self, subgraph: Any, prompt: str, num_queries: int = 3) -> List[str]:
        """Generuje zapytania do wyszukiwarki używając LLM."""
        # Przygotuj prompt dla LLM
        # TODO: Ulepszyć ekstrakcję informacji z subgraph
        subgraph_str = str(subgraph) # Prosta konwersja na string, można ulepszyć
        llm_prompt = f"""
        Na podstawie poniższego podgrafu wiedzy i głównego zapytania użytkownika, wygeneruj {num_queries} różnorodnych zapytań do ogólnej wyszukiwarki internetowej (np. Google).
        Zapytania powinny pomóc znaleźć ogólne informacje, kontekst, wiadomości i potencjalnie powiązane koncepcje.
        Format odpowiedzi: Każde zapytanie w nowej linii, bez numeracji.

        Podgraf:
        {subgraph_str}

        Główne zapytanie użytkownika:
        {prompt}

        Wygenerowane zapytania:
        """
        print("Generowanie zapytań dla WebSearch używając LLM...")
        raw_queries = generate_completion(llm_prompt)

        if raw_queries.startswith("Błąd:") or not raw_queries.strip():
             print(f"Nie udało się wygenerować zapytań przez LLM. Używanie domyślnego zapytania. Błąd: {raw_queries}")
             return [prompt] # Fallback

        # Przetwarzanie odpowiedzi LLM
        queries = [q.strip() for q in raw_queries.strip().split('\n') if q.strip()]
        print(f"Wygenerowane zapytania przez LLM: {queries}")
        return queries if queries else [prompt] # Zwróć wygenerowane lub fallback

    def search(self, subgraph: Any, prompt: str, max_results_per_query: int = 3) -> List[Dict[str, Any]]:
        """Przeszukuje internet używając Firecrawl z zapytaniami generowanymi przez LLM."""
        queries = self._generate_queries(subgraph, prompt)
        all_results = []

        for query in queries:
            # Użyj funkcji z api_clients
            results = search_firecrawl(query, fetch_content=True, max_results=max_results_per_query)
            all_results.extend(results)
            # Opcjonalnie: Zastosuj LLM do podsumowania/ekstrakcji z 'content' wyników, jeśli potrzebne
            # for result in results:
            #     if result.get('content'):
            #         summary_prompt = f"Podsumuj poniższy tekst w kontekście zapytania '{query}':\n\n{result['content'][:2000]}" # Ogranicz długość
            #         summary = generate_completion(summary_prompt)
            #         result['llm_summary'] = summary # Dodaj podsumowanie do wyniku

        total_results = len(all_results)
        print(f"WebSearchAgent zakończył pracę, zwracając {total_results} wyników.")
        # Można dodać logikę deduplikacji wyników, jeśli wiele zapytań zwraca te same URL
        # unique_results = {r['url']: r for r in all_results}.values()
        # print(f"Po deduplikacji pozostało {len(unique_results)} wyników.")
        # return list(unique_results)
        return all_results

In [71]:

from typing import List, Dict, Any, Optional
import re # Import re do prostego czyszczenia tekstu dla fallbacku

class SLSAgent:
    def __init__(self, use_pubmed_api: bool = True):
        self.use_pubmed_api = use_pubmed_api
        print(f"SLSAgent skonfigurowany do użycia PubMed API: {self.use_pubmed_api}")

    def _generate_scientific_queries(self, input_text: str, num_queries: int = 2) -> List[str]:
        """Generuje zapytania do wyszukiwarek naukowych używając LLM na podstawie tekstu wejściowego."""

        llm_prompt = f"""
                Na podstawie poniższego tekstu analitycznego, zidentyfikuj kluczowe pojęcia biomedyczne (np. z sekcji Definitions) i relacje (np. z sekcji Relationships). Następnie wygeneruj {num_queries} zapytań zoptymalizowanych dla wyszukiwarek literatury naukowej, takich jak PubMed. Używaj precyzyjnych terminów, operatorów logicznych (AND, OR, NOT) i ewentualnie formatowania specyficznego dla PubMed (np. [Title/Abstract], [MeSH Terms]). Skup się na znalezieniu relevantnych publikacji naukowych potwierdzających, rozwijających lub kwestionujących informacje z tekstu wejściowego.
                Format odpowiedzi: Każde zapytanie w nowej linii, bez numeracji.

                Tekst Wejściowy:
                ```
                {input_text}
                ```

                Wygenerowane zapytania (format PubMed):
        """
        print("Generowanie zapytań dla SLS używając LLM...")
        raw_queries = generate_completion(llm_prompt)

        # --- POPRAWIONA LOGIKA FALLBACK ---
        if raw_queries.startswith("Błąd:") or not raw_queries.strip():
             print(f"Nie udało się wygenerować zapytań naukowych przez LLM. Używanie fallbacku opartego na input_text. Błąd: {raw_queries}")
             # Prosty fallback: weź pierwsze ~10 słów z input_text jako zapytanie
             # Usuń znaki specjalne, weź unikalne słowa
             words = re.findall(r'\b\w+\b', input_text.lower())[:15] # Weź pierwsze 15 słów
             fallback_query = " ".join(list(dict.fromkeys(words))) # Unikalne słowa w kolejności
             print(f"Wygenerowano zapytanie fallback: {fallback_query}")
             # Zwróć listę z jednym zapytaniem fallback lub pustą listę, jeśli tekst był pusty
             return [fallback_query] if fallback_query else []

        queries = [q.strip() for q in raw_queries.strip().split('\n') if q.strip()]
        print(f"Wygenerowane zapytania naukowe przez LLM: {queries}")
        # --- POPRAWIONA WARTOŚĆ ZWRACANA ---
        # Jeśli LLM zwrócił pustą listę (mimo że nie było błędu), też użyj fallbacku
        if not queries:
             words = re.findall(r'\b\w+\b', input_text.lower())[:15]
             fallback_query = " ".join(list(dict.fromkeys(words)))
             print(f"LLM zwrócił puste zapytania. Wygenerowano zapytanie fallback: {fallback_query}")
             return [fallback_query] if fallback_query else []
        else:
             return queries


    def search_papers(self, input_text: str, max_results_per_query: int = 5) -> List[Dict[str, Any]]:
        """Wyszukuje artykuły używając PubMed API i/lub Firecrawl z zapytaniami LLM."""
        # --- POPRAWIONE WYWOŁANIE _generate_scientific_queries ---
        queries = self._generate_scientific_queries(input_text=input_text) # Przekazujemy input_text

        if not queries: # Jeśli generowanie zapytań (nawet fallback) zawiodło
             print("SLSAgent: Nie udało się wygenerować żadnych zapytań. Zwracam pustą listę.")
             return []

        all_articles = []
        pubmed_failed_or_disabled = not self.use_pubmed_api

        for query in queries:
            articles_pubmed = None
            # 1. Spróbuj PubMed Entrez, jeśli włączone
            if self.use_pubmed_api:
                print(f"SLSAgent: Wyszukiwanie w PubMed dla zapytania: '{query}'") # Dodano log
                articles_pubmed = search_pubmed_entrez(query, max_results=max_results_per_query)
                if articles_pubmed is not None:
                    all_articles.extend(articles_pubmed)
                    print(f"Dodano {len(articles_pubmed)} artykułów z PubMed dla zapytania '{query}'.")
                else:
                    print(f"PubMed Entrez zwrócił błąd dla zapytania '{query}'.")
                    pubmed_failed_or_disabled = True

            # 2. Użyj Firecrawl jako fallback
            if pubmed_failed_or_disabled or (self.use_pubmed_api and not articles_pubmed and articles_pubmed is not None):
                # Dodano warunek 'articles_pubmed is not None', aby nie uruchamiać Firecrawl, jeśli Entrez miał błąd, chyba że Entrez jest wyłączony
                 if not articles_pubmed and articles_pubmed is not None:
                      print(f"PubMed nie znalazł wyników dla '{query}'. Próba za pomocą Firecrawl.")
                 elif pubmed_failed_or_disabled and self.use_pubmed_api: # Jeśli Entrez miał błąd
                       print(f"Próba wyszukania naukowego za pomocą Firecrawl z powodu błędu Entrez dla zapytania: {query}")
                 elif not self.use_pubmed_api: # Jeśli Entrez był wyłączony
                        print(f"Próba wyszukania naukowego za pomocą Firecrawl (PubMed API wyłączone) dla zapytania: {query}")

                # firecrawl_query = f"{query} site:pubmed.ncbi.nlm.nih.gov OR site:scholar.google.com OR site:biorxiv.org OR site:medrxiv.org OR site:arxiv.org"
                # articles_firecrawl = search_firecrawl(firecrawl_query, fetch_content=False) # Domyślnie nie pobieraj treści
                # if articles_firecrawl:
                 #   for article in articles_firecrawl:
                 #       article['source'] = 'firecrawl_scientific_search'
                #    all_articles.extend(articles_firecrawl)
                #    print(f"Dodano {len(articles_firecrawl)} potencjalnych artykułów z Firecrawl dla zapytania '{query}'.")
                # else:
                #      print(f"Firecrawl nie znalazł wyników dla zapytania: '{firecrawl_query}'")


        total_articles = len(all_articles)
        print(f"SLSAgent zakończył pracę, zwracając łącznie {total_articles} potencjalnych artykułów.")
        # TODO: Deduplikacja wyników (np. na podstawie DOI lub tytułu)
        # TODO: Opcjonalna ocena relevancji przez LLM na podstawie abstraktów/snippetów
        return all_articles

In [72]:

class ContextBuilderAgent:
    def __init__(self):
        print("Inicjalizacja ContextBuilderAgent...")
        # Inicjalizacja specjalistycznych agentów (bez Aggregatora)
        self.web_searcher = WebSearchAgent()
        self.sls_searcher = SLSAgent(use_pubmed_api=True)
        # self.kg_constructor = KnowledgeGraphConstructorAgent()
        # Usunięto self.aggregator
        print("Agenci podrzędni (Web, SLS) zainicjalizowani.") # Usunięto Aggregator z logu

    # --- Metody skopiowane z ContextAggregatorAgent ---

    def _prepare_llm_prompt(self, context_data: Dict[str, Any]) -> str:
        """Przygotowuje prompt dla LLM na podstawie zebranych danych."""

        initial_text = context_data.get('initial_input_text', 'Brak tekstu wejściowego.')

        web_info_parts = []
        # Zakładamy, że web_search_results jest pustą listą, ale zostawiamy logikę na przyszłość
        for res in context_data.get('web_search_results', []):
            info = res.get('title') if res.get('title') else res.get('snippet')
            url = res.get('url', 'N/A')
            if info:
                web_info_parts.append(f"- {info} (URL: {url})")
        web_info_str = "\n".join(web_info_parts) if web_info_parts else "Brak wyników z wyszukiwania w internecie (funkcjonalność wyłączona lub brak wyników)." # Zaktualizowano komunikat

        literature_abstracts = []
        for res in context_data.get('scientific_literature_results', []):
            abstract = res.get('abstract')
            title = res.get('title')
            url = res.get('url', 'N/A')
            pmid = res.get('pmid', 'N/A')
            if abstract:
                # Używamy \n dla nowej linii w f-string, spacja może nie być potrzebna
                literature_abstracts.append(f"- {title} (PMID: {pmid}, URL: {url}):\n{abstract}")
            elif title:
                 literature_abstracts.append(f"- {title} (PMID: {pmid}, URL: {url}) (brak abstraktu)")
        literature_info_str = "\n\n".join(literature_abstracts) if literature_abstracts else "Brak wyników z literatury naukowej."

        llm_prompt = f"""
        Zadanie: Jesteś specjalistą analizującym dane biomedyczne. Na podstawie poniższego **Tekstu Wejściowego** oraz dodatkowych informacji znalezionych w **Internecie** i **Literaturze Naukowej**, wygeneruj szczegółowe, ustrukturyzowane i obiektywne podsumowanie kontekstu w języku angielskim. Zachowaj neutralny ton. Odpowiedź powinna ściśle trzymać się dostarczonych informacji i mieć następującą strukturę:

        **1. Analizowany Temat:**
        (Krótkie, 1-2 zdaniowe podsumowanie głównych pojęć i relacji opisanych w Tekście Wejściowym.)

        **2. Kluczowe Definicje:**
        (Wylistuj definicje pojęć zawarte w Tekście Wejściowym. Jeśli definicji brakuje w tekście wejściowym, ale została znaleziona w źródłach, możesz ją dodać z adnotacją źródła, np. [wg. PubMed: ID].)
        - Pojęcie 1: Definicja...
        - Pojęcie 2: Definicja...
        - ...

        **3. Główne Powiązania/Mechanizmy Opisane w Tekście Wejściowym:**
        (Wylistuj i opisz zwięźle relacje wspomniane w Tekście Wejściowym. Możesz wzbogacić opis o 1-2 zdania kontekstu z znalezionych źródeł, jeśli bezpośrednio dotyczą tej relacji.)
        - Relacja 1 (np. Zapalenie zwiększa Amyloid Beta): Opis relacji... (ewentualne wzbogacenie ze źródeł)
        - Relacja 2 (np. Amyloid Beta akumuluje się w Ch. Alzheimera): Opis...
        - ...

        **4. Kontekst z Literatury Naukowej:**
        (Syntetyczny przegląd najważniejszych informacji i wniosków z abstraktów naukowych. Pogrupuj informacje tematycznie, jeśli to możliwe. Unikaj bezpośredniego cytowania poszczególnych artykułów, skup się na zagregowanej wiedzy.)

        **5. Kontekst z Ogólnego Internetu:**
        (Podsumowanie relevantnych informacji z tytułów/snippetów znalezionych w Internecie. Wskaż główne punkty lub perspektywy. Pomiń tę sekcję, jeśli brak wyników lub informacji.)

        **6. Synteza i Wnioski Ogólne:**
        (Krótkie, 2-4 zdaniowe podsumowanie łączące wszystkie zebrane informacje. Jakie są główne wnioski płynące z analizy tekstu wejściowego w świetle znalezionego kontekstu?)

        --- Dane Wejściowe dla LLM ---

        **Tekst Wejściowy do Analizy:**
        ```
        {initial_text}
        ```

        **Informacje z Internetu (Tytuły/Fragmenty):**
        ```
        {web_info_str}
        ```

        **Informacje z Literatury Naukowej (Tytuły/Abstrakty):**
        ```
        {literature_info_str}
        ```

        --- Koniec Danych Wejściowych ---

        Wygenerowane Podsumowanie Kontekstu (w formacie opisanym powyżej):
        """
        return llm_prompt

    def generate_aggregated_context(self, context_data: Dict[str, Any]) -> str:
        """Generuje zagregowany kontekst tekstowy używając LLM."""
        print("ContextBuilderAgent: Generowanie zagregowanego kontekstu...") # Zmieniono nazwę agenta w logu

        has_literature_results = bool(context_data.get('scientific_literature_results'))
        # Można też sprawdzić web_search_results, jeśli zostaną włączone i naprawione
        # has_web_results = bool(context_data.get('web_search_results'))
        # if not has_literature_results and not has_web_results:

        if not has_literature_results: # Na razie sprawdzamy tylko literaturę
             print("Ostrzeżenie (ContextBuilderAgent): Brak wyników z literatury. Zwracam podstawowy kontekst.")
             return f"Analiza tekstu wejściowego wykazała następujące pojęcia i relacje:\n{context_data.get('initial_input_text', '')}\nNie znaleziono dodatkowego kontekstu w dostępnych źródłach (Literatura Naukowa, Internet)."

        llm_prompt = self._prepare_llm_prompt(context_data)
        print("\n--- Prompt dla generowania zagregowanego kontekstu ---") # Zmieniono nazwę logu
        print(llm_prompt)
        print("--- Koniec Promptu dla generowania zagregowanego kontekstu ---\n")

        # Wywołujemy generate_completion bezpośrednio
        aggregated_context = generate_completion(llm_prompt, model_preference="gemini", max_tokens=3000)

        if aggregated_context.startswith("Błąd:") or not aggregated_context.strip():
            print(f"Błąd (ContextBuilderAgent): LLM nie zwrócił poprawnego kontekstu. Błąd: {aggregated_context}")
            return f"Wystąpił błąd podczas generowania zagregowanego kontekstu przez LLM: {aggregated_context}"
        else:
            print("ContextBuilderAgent: Pomyślnie wygenerowano zagregowany kontekst.")
            return aggregated_context.strip()

    # --- Główna metoda build_context ---

    def build_context(self, input_text: str) -> Dict[str, Any]:
        """Buduje kontekst, w tym zagregowane podsumowanie tekstowe."""
        print("\n=== ContextBuilderAgent: Rozpoczynanie budowania kontekstu ===")

        # Web search jest wyłączony zgodnie z ustaleniami
        # print("\n--- Uruchamianie WebSearchAgent ---")
        # web_results = self.web_searcher.search(input_text=input_text)
        web_results = []
        print(f"ContextBuilderAgent: Pominięto WebSearchAgent (web_results ustawione na pustą listę).")

        print("\n--- Uruchamianie SLSAgent ---")
        scientific_results = self.sls_searcher.search_papers(input_text=input_text)
        print(f"ContextBuilderAgent: Otrzymano {len(scientific_results)} wyników z SLSAgent.")

        # --- Tutaj można dodać wywołanie KG Agenta ---

        # Stworzenie wstępnego kontekstu
        context = {
            "initial_input_text": input_text,
            #"web_search_results": web_results, # Pusta lista
            "scientific_literature_results": scientific_results,
            # "knowledge_graph": kg_results
        }

        # --- Generowanie Zagregowanego Kontekstu (bezpośrednie wywołanie metody) ---
        # print("\n--- Uruchamianie ContextAggregatorAgent ---") # Usunięto log o osobnym agencie
        # Zamiast: aggregated_context_text = self.aggregator.generate_aggregated_context(context)
        # Używamy:
        aggregated_context_text = self.generate_aggregated_context(context) # <--- BEZPOŚREDNIE WYWOŁANIE
        context['aggregated_context'] = aggregated_context_text # Dodajemy wynik do słownika

        print("\n=== ContextBuilderAgent: Zakończono budowanie kontekstu (z agregacją) ===")
        return context

In [76]:
import json

def run_context_building():
    """Główna funkcja uruchamiaj�ca proces budowania kontekstu."""

    print("="*50)
    print(" Rozpoczynanie procesu budowania kontekstu")
    print("="*50)

    # Przyk�adowy subgraph i prompt
   # example_subgraph = {
   #     "nodes": [{"id": "1", "name": "Chronic Kidney Disease"}, {"id": "2", "name": "Anemia"}],
   #     "edges": [{"source": "1", "target": "2", "relation": "associated_with"}]
   # }
    #with open('/home/wiktor/hackathon/hackathon/sample_subgraph.json', 'r') as f:
    #    example_subgraph = json.load(f)

    #example_prompt = "Investigate how inflammation influences the development of Alzheimer�s disease by analyzing the mechanisms related to amyloid beta accumulation, the role of tau protein, and microglial activation, as illustrated in the provided knowledge graph."

    analysis_text = """# Ontologist Analysis
                    {
                    ### Definitions:
                    - **Inflammation**: A biological response of body tissues to harmful stimuli, such as pathogens, damaged cells, or irritants. It involves immune cell activation, blood vessel dilation, and the release of various molecules that promote healing.
                    - **Amyloid Beta**: A peptide that is produced in the brain through the breakdown of a larger protein called amyloid precursor protein (APP). Accumulation of amyloid beta in the brain is commonly associated with neurodegenerative diseases, especially Alzheimer's disease.
                    - **Alzheimer's Disease**: A progressive neurological disorder characterized by cognitive decline, memory loss, and changes in behavior and personality. It is associated with the accumulation of amyloid beta plaques and neurofibrillary tangles in the brain.

                    ### Relationships
                    - **Inflammation increases Amyloid Beta**: This relationship indicates that the presence of inflammation in the brain can lead to an increase in the production or aggregation of amyloid beta peptides. This connection suggests that inflammatory processes might play a crucial role in the pathogenesis of Alzheimer's disease by enhancing the toxic effects of amyloid beta.
                    ###
                    - **Amyloid Beta accumulates in Alzheimer's Disease**: This relationship illustrates how the accumulation of amyloid beta is a defining feature of Alzheimer's disease. In patients with Alzheimer�s, there is a significant build-up of amyloid beta plaques in the brain, which is believed to contribute to the neurodegenerative processes observed in the disorder, such as synaptic dysfunction and neuronal death.
                    }"""

    print("\nDane wej�ciowe:")
    #print(f"  Subgraph: {json.dumps(example_subgraph, indent=2)}")
    #print(f"  Prompt: {example_prompt}")
    print(f" Tekst wej�ciowy analizy: {analysis_text}")
    # Utworzenie i uruchomienie g��wnego agenta
    context_builder = ContextBuilderAgent()
    final_context = context_builder.build_context(input_text=analysis_text)

    print("\n" + "="*50)
    print(" Finalny Kontekst Zosta� Zbudowany")
    print("="*50)

    # Wy�wietlenie podsumowania wynik�w
    print(f"Liczba wynik�w z wyszukiwania web: {len(final_context.get('web_search_results', []))}")
    print(f"Liczba wynik�w z literatury naukowej: {len(final_context.get('scientific_literature_results', []))}")

    # Opcjonalnie: Zapisz kontekst do pliku JSON
    try:
        output_filename = "final_context.json"
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(final_context, f, indent=4, ensure_ascii=False)
        print(f"\nPe�ny kontekst zapisano do pliku: {output_filename}")
    except Exception as e:
        print(f"\nB��d podczas zapisywania kontekstu do pliku: {e}")

    # Mo�esz tutaj doda� kod do dalszego przetwarzania `final_context`

if __name__ == "__main__":
    run_context_building()

 Rozpoczynanie procesu budowania kontekstu

Dane wej�ciowe:
 Tekst wej�ciowy analizy: # Ontologist Analysis
                    {
                    ### Definitions:
                    - **Inflammation**: A biological response of body tissues to harmful stimuli, such as pathogens, damaged cells, or irritants. It involves immune cell activation, blood vessel dilation, and the release of various molecules that promote healing.
                    - **Amyloid Beta**: A peptide that is produced in the brain through the breakdown of a larger protein called amyloid precursor protein (APP). Accumulation of amyloid beta in the brain is commonly associated with neurodegenerative diseases, especially Alzheimer's disease.
                    - **Alzheimer's Disease**: A progressive neurological disorder characterized by cognitive decline, memory loss, and changes in behavior and personality. It is associated with the accumulation of amyloid beta plaques and neurofibrillary tangles in the brain.